In [ ]:
# trys to catch if notebook is run within the browser
from IPython import get_ipython
if get_ipython().__class__.__name__ == 'Interpreter':
    import piplite
    await piplite.install('matplotlib')
    await piplite.install('ase')

    await piplite.install('widget_code_input')
    await piplite.install('chemiscope')
    await piplite.install('https://agoscinski.github.io/scicode-widgets-wheels/dist/scwidgets-0.0.0.dev0-py3-none-any.whl')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Layout, Output, Textarea, HTML, HBox
from scwidgets import (AnswerRegistry, TextareaAnswer, CodeDemo,
                       ParametersBox, PyplotOutput, ClearedOutput,
                       AnimationOutput,CheckRegistry,Answer)
import ase
from ase.io import read, write
import itertools
import functools    
from copy import deepcopy

In [ ]:
# this is a temporary hack until the latest chemiscope release containts the merged fix
if get_ipython().__class__.__name__ == 'Interpreter':
    chemiscope.jupyter._is_running_in_notebook = lambda : True

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html
<style>
.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea  {  height:auto !important;
    max-height: 5000px; overflow-y: hidden }
</style>
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

Please enter your name as `SurnameName` to initialize the answer file. 

In [ ]:
#check_registry = CheckRegistry() 
answer_registry = AnswerRegistry(prefix="module_02")
display(answer_registry)

In [ ]:
def plot_lattice(ax, a1, a2, basis=None, alphas=None, s=20, c='red', 
                 lattice_size = 60, head_length = 0.5, head_width= 0.2, width=0.05):
    if basis is None:
        basis = np.array([[0,0]])
    A = np.array([a1, a2])
    # each atom in the basis gets a different basis alpha value when plotted
    if alphas is None:
        alphas = np.linspace(1, 0.3, len(basis))
    for i in range(len(basis)):
        lattice = (np.mgrid[:lattice_size,:lattice_size].T @ A + basis[i]).reshape(-1, 2)
        lattice -= (np.array([lattice_size//2,lattice_size//2]) @ A).reshape(-1, 2)
        ax.scatter(lattice[:,0], lattice[:,1], color=c, s=s, alpha=alphas[i])
        
    ax.fill([0,a1[0],(a1+a2)[0],a2[0]], [0,a1[1],(a1+a2)[1],a2[1]], color=c, alpha=0.2)
    ax.arrow(0,0, a1[0], a1[1],width=width,
             length_includes_head=True,
             fc=c, ec='black')
    ax.arrow(0,0, a2[0], a2[1],width=width,
             length_includes_head=True,
             fc=c, ec='black')    

In [ ]:
def reciprocal_lattice_vectors(a1, a2):
    # compute it in obfuscated way
    reciprocal_lattice = 2*np.pi*ase.Atoms(cell=[[a1[0], a1[1], 0], [a2[0], a2[1], 0], [0,0,1]]).cell.reciprocal()
    return reciprocal_lattice[0][:2], reciprocal_lattice[1][:2]


# Diffraction from an atomic structure

An actual crystalline structure involves both a lattice and of an a-periodic basis $\{\mathbf{s}_i\}$ of atoms. Each will scatter with a form factor $f_i$ (that depends on the modulus of $\mathbf{k}$ but we will take to be a constant proportional to the atomic charge $Z_i$). The scattered amplitude can be written as  

$$
F(\mathbf{k}) = \frac{1}{N} \sum^N_{j \textrm{ atom}} f_j\exp(-\mathrm{i}\mathbf{k}\cdot\mathbf{r}_j)\textrm{, with }\mathbf{r}_j\textrm{ position of atom }j
$$

by breaking the position of atoms into lattice vector and basis positions ($\mathbf{r}_j=\mathbf{T}+\mathbf{s}_m$), we can write 

$$
F(\mathbf{k}) = \frac{1}{n_\mathrm{cell} n_\mathrm{basis}} 
\sum^{n_\mathrm{cell}}_{\mathbf{T}} \exp(-\mathrm{i}\mathbf{k}\cdot \mathbf{T}) 
\sum_m^{n_\mathrm{basis}} f_m \exp(-\mathrm{i}\mathbf{k}\cdot\mathbf{s}_m).
$$

One sees that the sum over lattice vectors selects the diffracted wavevectors that correspond to a reciprocal lattice vector, $\mathbf{k}=\mathbf{G}$, as we saw in the previous section. The sum over the atomic basis modulates the amplitude of the peak through a _structure factor_, which we will see encodes information on the type and position of atoms within the unit cell

$$
F_\mathbf{G} = \sum_m^{n_\mathrm{basis}} f_m \exp(-\mathrm{i}\mathbf{G}\cdot\mathbf{s}_m).
$$



[comment]: <> (<img src="figures/diffraction.png" width="600"/>)

In the scattering geometry above, one can determine the conditions for scattering in terms of the scattering angle $2\theta$, as follows. If $k$ is the modulus of both $\mathbf{k}_\mathrm{in}$ and $\mathbf{k}_\mathrm{out}$, and $\mathbf{G} = \mathbf{k} = \mathbf{k}_\mathrm{in}-\mathbf{k}_\mathrm{out}$ must hold, a first necessary condition is

$$
G^2 = |\mathbf{k}_\mathrm{in}-\mathbf{k}_\mathrm{out}|^2 = 2k^2 (1-\cos 2\theta) = 4k^2 sin^2 \theta
$$

hence, $\sin\theta=G/2k$. To determine the orientation of $\mathbf{G}$ a second condition is needed. We can consider the angle between $\mathbf{G}$ and the incoming vector $\phi$, that can be set by changing the orientation of the crystal. Thus, by writing $|\mathbf{G}-\mathbf{k}_\mathrm{in}|^2 = |\mathbf{k}_\mathrm{out}|^2$ we can easily get $G = 2 k \cos\phi$, and hence $\cos\phi = \sin\theta$. If the sample is formed by uniformly oriented grains, the second condition is always satisfied by some crystals, and so the diffraction pattern is just a sequence of peaks at particular values of the angle $2\theta$.  The intensity of each peak is given by the square modulus of the structure factor, $|F_\mathbf{G}|^2$.

This widget below computes the powder (rotationally averaged) diffraction pattern for a crystal with a basis of two atoms. The function computes the list of diffraction peaks, with the corresponding structure factor. 
It is not entirely trivial, and so it is already implemented: you don't have to change it, but you can read it and understand what it does. Experiment with the widget, and then move to the exercises below, that will ask you to comment on what you observe in different scenarios. 

Parameters are as follows:
* $a_{ij}$: components of the lattice vectors
* $\phi$: rigid rotation of the lattice (does it have an impact on the diffraction?)
* $s_{1,2}$: fractional coordinates of the second atom of the basis (first atom is in (0,0))
* $f_{1,2}$: atomic form factors for the two atoms in the basis (roughly take it to be the atomic number)
* $\lambda$: wavelength of the scattering radiation

In [ ]:
# set upt the code widget window
ex12_wci = WidgetCodeInput(
        function_name="diffraction_peaks",
        function_parameters="basis, atomic_ff, reciprocal_b1, reciprocal_b2, wavelength",
        docstring="""
Computes the list of peaks for a lattice with a given (real-space) basis 
and reciprocal lattice, and for a given wavelength of the incoming radiation.

:param basis: list of N 2D vectors corresponding to the (real space!) position of the basis atoms
:param atomic_form_factors: atomic form factors of atoms in lattice, array of length N
:param reciprocal_b1, reciprocal_b2:  reciprocal lattice vectors
:param wavelength: wavelength of the incoming radiation

:return: The list of diffraction peaks, as [h, l, theta, intensity]
""",
    function_body="""    
import numpy as np

def compute_absolute_structure_factor(sj, fj, G):
    # sj: atomic basis (n_basis x 2)
    # fj: form factors (n_basis)
    # G: reciprocal lattice vectors (2D)
    return np.abs(fj @ np.exp(-1j * sj @ G))

# wave number (modulus of the incoming wavevector)
k = np.pi*2/wavelength 
# determine the range of reciprocal lattice vectors that could give rise to permissible reflections
if reciprocal_b1@reciprocal_b1 != 0:
    n1 = int((k*2)/np.sqrt(reciprocal_b1@reciprocal_b1))+1
else:
    n1 = 0
if reciprocal_b2@reciprocal_b2 != 0:
    n2 = int((k*2)/np.sqrt(reciprocal_b2@reciprocal_b2))+1  
else:
    n2 = 0

# allocated space for the list of peaks
lpeaks = []
for v1 in range(-n1,n1+1):
    for v2 in range(-n2,n2+1):
        # reciprocal lattice vector
        G = reciprocal_b1*v1 + reciprocal_b2*v2

        # theta (from 2theta geometry)
        sin_theta = np.sqrt(G@G)/(2*k)
        if sin_theta > 1: # discards reflections that fall outside of the permissible range
            continue
        theta = np.arcsin(sin_theta)
        # structure factor
        absolute_structure_factor = compute_absolute_structure_factor(basis, atomic_ff, G)
        lpeaks.append([v1, v2, theta, absolute_structure_factor**2])
return np.asarray(lpeaks)
"""
        )


In [ ]:
def plot_diffraction(a11, a12, a21, a22, s1, s2, f1, f2, phi, wavelength,code_input,visualizers):
    print_output = visualizers[0]
    pyplot_output = visualizers[1]
    table_output = visualizers[2]
    axes = pyplot_output.figure.get_axes()
    def rot2d(angle):
        return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    rot_phi = rot2d(phi)
    a1 = np.array([a11, a12]) @ rot_phi
    a2 = np.array([a21, a22]) @ rot_phi
    basis = np.asarray([0*a1,s1*a1+s2*a2])
    plot_lattice(axes[0], a1, a2, basis=basis, alphas=[f1/40, f2/40], c='red')
    axes[0].set_title('real space')
    axes[0].set_xlim(-5,5)
    axes[0].set_ylim(-5,5)
    axes[0].set_xlabel("$x$ / Å")
    axes[0].set_ylabel("$y$ / Å")
    
    b1, b2 = reciprocal_lattice_vectors(a1, a2)
    
    dpeaks = code_input.get_function_object()( basis, np.asarray([f1, f2]), b1, b2, wavelength )
    
    twotheta_grid = np.linspace(0, 180, 720)
    dp_grid = np.zeros(len(twotheta_grid))
    for _, _, t, f2 in dpeaks:
        dp_grid += np.exp(-(twotheta_grid-2*t*180/np.pi)**2/0.5)*f2
    
    axes[1].clear()
    # we plot two theta
    axes[1].plot(twotheta_grid, dp_grid, 'b-')
    
    axes[1].set_xlim(0,180)
    axes[1].set_xlabel("$2\\theta$ / degree °")
    axes[1].set_title('Diffraction pattern')
    axes[1].set_ylabel("Intensity $|F(\mathbf{k}(\\theta))|$")
    
    axes[0].set_aspect('equal')
    axes[1].set_aspect(150/np.max(dp_grid))
    with table_output:
        header = """
                      v1 / Å 
                      v2 / Å 
                      2θ / degree ° 
                      |FG|2 
                    """
        # cleans up peak info for displaying
        tpeaks = []
        for d in dpeaks[np.argsort(dpeaks[:,2])]:
            tpeaks.append( [ int(d[0]), int(d[1]), np.round(2*d[2]*180/np.pi,2),  np.round(d[3],1) ])
        reflection_table_html.value = array_to_html_table(tpeaks, header)
        display(reflection_table)
        
def array_to_html_table(numpy_array, header):
    rows = ""
    for i in range(len(numpy_array)):
        rows += "<tr>" + functools.reduce(lambda x,y: x+y,
                             map(lambda x: "<td>" + str(x) + "</td>",
                                 numpy_array[i])
                            ) + "</tr>"

    return "<table>" + header + rows + "</table>"
        

reflection_table_html = HTML(
    value=f"dpeaks")

reflection_table = HBox(layout=Layout(width='99%', height='250px', overflow_y='auto'))
reflection_table.children += (reflection_table_html,)

In [ ]:
diffraction_figure, _ = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
diffraction_output = PyplotOutput(diffraction_figure)

ex12_wp = ParametersBox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
                            a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
                            a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
                            a22 = (1.5, -4, 4, 0.1, r'$a_{22} / Å$'),
                            s1 = (0.25, 0.01, 0.99, 0.01, r'$s_1$'),
                            s2 = (0.75, 0.01, 0.99, 0.01, r'$s_2$'),
                            f1 = (10., 1., 40., 1, r'$f_{1}$'),
                            f2 = (30., 0, 40., 1, r'$f_{2}$'),
                            phi = (0., 0., 2*np.pi, 0.1, r'$\phi$'),
                            wavelength = (0.1, 1.0, 2, 0.05, r'$\lambda$'),
                            refresh_mode="click")
ex12_code_demo = CodeDemo(
            input_parameters_box=ex12_wp,
            code_input= ex12_wci,
            visualizers = [ClearedOutput(),diffraction_output,ClearedOutput()],
            update_visualizers = plot_diffraction
)

answer_registry.register_answer_widget("ex12-function", ex12_code_demo)
display(ex12_code_demo)
ex12_code_demo.run_demo()

Set $f_2$ to zero (so that effectively this becomes a lattice with a single atom) and set the unit cell to be a $1\times 1.2$ rectangle. Set the wavelength to 1. Observe how the position and intensity of the peaks change when you change the dimensions of the lattice.


<span style="color:blue"> **12a** What happens if you set the off-diagonal term $a_{12}$ to be (slightly) different from zero? What happens if you set the two lattice vectors to be orthogonal and equal in length?
</span>

In [ ]:
ex012a_txt = TextareaAnswer("Enter your answer here")
answer_registry.register_answer_widget("ex12a-answer", ex012a_txt)
display(ex012a_txt)